# cellPT embedding extraction


paper can be found in 
https://www.biorxiv.org/content/10.1101/2023.10.03.560734v1.full 
the data code section point to 
github in https://github.com/OmicsML/CellPLM
There, the the readme at 
https://github.com/OmicsML/CellPLM/blob/main/ckpt/README.md 
directs us to the dropBox folder for down loading the check-point:
https://www.dropbox.com/scl/fo/i5rmxgtqzg7iykt2e9uqm/h?rlkey=o8hi0xads9ol07o48jdityzv1&e=1&dl=0
and this is where the models have been downloaded from



In [1]:
import json

import pandas as pd
import torch

Where was the model saved?

In [2]:
model_dict = "."
model_file = f"{model_dict}/20230926_85M.best.ckpt"
model_config_file = f"{model_dict}/20230926_85M.config.json"

Load the config and the model

In [3]:
with open(model_config_file) as f:
    model_configs = json.load(f)

model = torch.load(model_file, map_location=torch.device("cpu"))

read the gene list from the config

In [4]:
gene_list = model_configs["gene_list"]
len(gene_list)

19374

Find the name of the encoding layer

In [5]:
for f in model["model_state_dict"].keys():
    s = model["model_state_dict"][f].shape
    if len(gene_list) in s:
        print(f"{f} - {s}")

embedder.feat_enc.emb - torch.Size([19374, 1024])
decoder.out_layer.dec_disp - torch.Size([19374])
decoder.out_layer.dec_mean.0.weight - torch.Size([19374, 128])
decoder.out_layer.dec_mean.0.bias - torch.Size([19374])


The encoding layer has a shape of `[num genes, embedding width]`.

The number of genes is the length of the list of gene names, so 19374

the enbedding with here is 1024

The name of the embedding layer is therefor `embedder.feat_enc.emb

In [6]:
embedding_layer = "embedder.feat_enc.emb"

Verify

In [7]:
model["model_state_dict"][embedding_layer].shape

torch.Size([19374, 1024])

load the tensor

In [8]:
encodings_torch = model["model_state_dict"][embedding_layer]

Convert to numpy format

In [9]:
encodings_numpy = encodings_torch.numpy()

Convert to pandas datafreame and attach the gene names as index 

In [10]:
encodings_df = pd.DataFrame(encodings_numpy, index=gene_list)

Take a peek at the encodings we got:

In [11]:
encodings_df.iloc[0:3]

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
ENSG00000000003,-0.008848,0.013859,-0.027063,-0.104422,0.052641,0.065944,-0.047289,0.059768,0.010998,0.046967,...,0.035736,-0.035231,0.021889,-0.051488,0.035529,0.028316,0.111450,-0.058094,-0.107816,0.034028
ENSG00000000005,0.100041,0.072579,0.082818,-0.041926,-0.002070,0.192488,-0.123851,0.168363,0.010028,0.023776,...,0.149555,0.039148,0.096405,0.031752,0.053479,-0.113100,-0.028484,-0.002721,-0.163965,0.090251
ENSG00000000419,0.051549,-0.039285,-0.037895,0.017505,0.033906,0.033948,-0.010280,-0.037145,-0.043533,-0.026587,...,0.021369,-0.044248,0.024618,0.002664,-0.094070,0.038673,0.034113,0.038242,0.099450,0.041229


looks right

save to a csv file (this may be a little slow, but probably less then a minute)

In [12]:
output_file_name = "./20230926_85M_encoding.csv"
encodings_df.to_csv(output_file_name)

And we are done.